# View Equipamentos Críticos

Equipamentos com maior downtime ou menor OEE.

In [ ]:
spark.sql(f"""
CREATE OR REPLACE VIEW {fqtn(SCHEMA_GOLD, 'vw_equipamentos_criticos')} AS
WITH equipment_metrics AS (
    SELECT 
        e.equipment_id,
        e.equipment_name,
        sum(f.downtime_minutes) as total_downtime,
        count(DISTINCT f.maintenance_sk) as maintenance_count,
        sum(q.failed_quantity) as total_failures
    FROM {fqtn(SCHEMA_GOLD, 'dim_equipamento')} e
    LEFT JOIN {fqtn(SCHEMA_GOLD, 'fact_manutencao')} f ON e.equipment_id = f.equipment_id
    LEFT JOIN {fqtn(SCHEMA_GOLD, 'fact_qualidade')} q ON e.equipment_id = q.equipment_id
    GROUP BY 1, 2
)
SELECT 
    equipment_id,
    equipment_name,
    total_downtime,
    maintenance_count,
    total_failures,
    (total_downtime + COALESCE(total_failures, 0) * 10) as criticality_score
FROM equipment_metrics
ORDER BY criticality_score DESC
""")